In [75]:
import pandas as pd
import statsmodels.formula.api as smf

def build_mixed_model(df, outcomes_dict, predictor_cols, outcome_label, covariate_cols=None):
    """
    Builds a mixed linear model using the provided dataframe, outcomes dictionary, predictors, outcome label, and optional covariates.
    
    Parameters:
    - df: pd.DataFrame - Original dataframe (a copy will be used).
    - outcomes_dict: dict[str, int/float] - Dictionary where keys are outcome column names (str),
      and values are corresponding timepoints (int/float).
    - predictor_cols: list[str] - List of predictor column names.
    - outcome_label: str - The label for the outcome variable to be visible in the model summary.
    - covariate_cols: list[str], optional - List of potential covariate column names to include as additional fixed effects. Default is None.
    
    Returns:
    - str: Summary of the fitted mixed linear model.
    """
    if covariate_cols is None:
        covariate_cols = []
    
    # Extract outcome_cols and timepoints from dict
    outcome_cols = list(outcomes_dict.keys())
    timepoints = list(outcomes_dict.values())
    
    if len(outcome_cols) != len(timepoints):
        raise ValueError("Outcomes dictionary must have matching keys and values.")
    
    # Make a copy of df
    ldf = df.copy()
    
    # Create or use subject_id
    ldf['subject_id'] = df.get('unique_id', pd.Series(range(len(df))))
    
    # Convert to numeric
    ldf = ldf.apply(pd.to_numeric, errors='coerce')
    
    # Reshape to long format
    df_long = pd.melt(
        ldf,
        id_vars=['subject_id'] + predictor_cols + covariate_cols,
        value_vars=outcome_cols,
        var_name='timepoint_str',  # Temporary name to avoid conflict
        value_name=outcome_label  # Use outcome_label here for visibility in summary
    )
    
    # Map timepoints using outcomes_dict
    df_long['timepoint'] = df_long['timepoint_str'].map(outcomes_dict)
    df_long = df_long.drop(columns=['timepoint_str'])  # Drop temporary column
    print(f'Input dataframe shape: {df_long.shape}')
    
    # Define model columns for dropna
    model_cols = [outcome_label, 'timepoint'] + predictor_cols + covariate_cols
    
    # Drop missing values
    df_long = df_long.dropna(subset=model_cols).reset_index(drop=True)
    print(f"Input dataframe shape after dropping missing values: {df_long.shape}")
    
    # Ensure subject_id is integer
    df_long['subject_id'] = df_long['subject_id'].astype(int)
    
    # Formula: outcome_label ~ timepoint + predictors + covariates
    all_fixed_effects = predictor_cols + covariate_cols
    formula = f'{outcome_label} ~ timepoint + ' + ' + '.join(all_fixed_effects)
    
    # Fit the model
    model = smf.mixedlm(formula, df_long, groups=df_long['subject_id'].values)
    result = model.fit()
    
    # Return the summary as string
    print(result.summary())  # Use as_text() to return as string
    #return result, df_long

In [77]:
def percent_non_null(df, columns):
    """
    Returns and prints the percentage of rows with non-null values 
    across the specified columns, and always shows per-column diagnostics.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    columns (list of str): Column names to check.

    Returns:
    float: Percentage of rows fully non-null across the given columns (rounded).
    """
    assert isinstance(columns, list), "columns must be a list of strings"

    total_rows = len(df)
    non_null_rows = df[columns].dropna().shape[0]
    percent = (non_null_rows / total_rows) * 100
    rounded_percent = round(percent, 2)

    if len(columns) == 1:
        print(f"{rounded_percent:.2f}% rows are non-null for the column")
    else:
        print(f"{rounded_percent:.2f}% rows are fully non-null across all specified columns")

    # Always show per-column completeness
    print("\nPer-column non-null percentages:\n")
    for col in columns:
        col_percent = df[col].notna().mean() * 100
        print(f"{col:35}: {col_percent:6.2f}%")

    return rounded_percent

In [79]:
# Import necessary libraries
import pandas as pd
import seaborn as sns 

# Load the variable-to-category mapping file
categories_mapping = pd.read_excel('categorized_variables_v3.xlsx')

# Load the main dataset
df = pd.read_csv('ForKathryn_2025Apr07-2_nopsw-1.csv')

# Replace common placeholders for missing values with None (NaN)
df.replace(['#NULL!', 'NULL', 'null', 'N/A', 'NA', '999', 999, '-999', '-999.00'], None, inplace=True)

/var/folders/zw/7l6f7k_j48jg8lp15zn3tmr00000gn/T/ipykernel_52549/2974152139.py:9: DtypeWarning: Columns (3,7,9,20,21,22,23,24,26,27,29,34,38,42,46,47,49,50,53,109,128,181,321,327,609,637,638,764,792,793,954,1036,1093,1107,1158,1201,1218,1237,1240,1242,1297,1311,1405,1422,1441,1444,1446,1449,1450,1454,1457,1458,1465,1466,1473,1476,1485,1486,1490,1494,1497,1498,1512) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ForKathryn_2025Apr07-2_nopsw-1.csv')


## Testing a Hypothesis
  

In [81]:
predictors = predictor_cols = [
    'hx_before_preg___1',              # History of anxiety before pregnancy
    'hx_before_preg___4',              # History of depression before pregnancy
    'hx_during_preg___1',              # Experience of anxiety during pregnancy
    'hx_during_preg___4',              # Experience of depression during pregnancy
    'EPDS_T1',                         # Maternal EPDS at first study questionniare
    'Panx_t_T1',                       # PROMIS anxiety 7a - Adult v1.0 T-scores at first study questionniare)
    'Zdistress_thermometer_combined',  # CZ-scores for Distress Thermometer combining English and French
    'SLPimpair_rawT1',                 # PROMIS sleep impairment Short Form 4a T-scores at first study questionniare
    'Anger_t_T1',                      # PROMIS Anger 5a T-scores at first study questionniare
    'CDrisc_T1'                        # Connor-Davidson Resilience Scale 2 (CD_RISC 2) at first study questionniare
]

outcomes = {'Total_M3_01' : 3, 'TotalBISQ_M12_01' : 12}
covariates = ['Mage','ses','Gestation']


In [87]:
percent_non_null(df, predictors + list(outcomes.keys()) + covariates)

20.55% rows are fully non-null across all specified columns

Per-column non-null percentages:

hx_before_preg___1                 :  95.45%
hx_before_preg___4                 :  95.45%
hx_during_preg___1                 :  95.35%
hx_during_preg___4                 :  95.35%
EPDS_T1                            :  88.57%
Panx_t_T1                          :  88.26%
Zdistress_thermometer_combined     :  88.64%
SLPimpair_rawT1                    :  79.12%
Anger_t_T1                         :  60.73%
CDrisc_T1                          :  78.51%
Total_M3_01                        :  33.07%
TotalBISQ_M12_01                   :  36.76%
Mage                               :  98.67%
ses                                :  98.03%
Gestation                          :  99.17%


20.55

In [89]:
build_mixed_model(df = df, 
                  outcomes_dict = outcomes,
                  predictor_cols = predictors,
                  covariate_cols = covariates,
                  outcome_label = 'TotalBISQ')            

Input dataframe shape: (21708, 16)
Input dataframe shape after dropping missing values: (5704, 16)
                  Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      TotalBISQ  
No. Observations:       5704         Method:                  REML       
No. Groups:             3474         Scale:                   132.7501   
Min. group size:        1            Log-Likelihood:          -22868.3262
Max. group size:        2            Converged:               Yes        
Mean group size:        1.6                                              
-------------------------------------------------------------------------
                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                      85.596    3.257 26.279 0.000 79.212 91.980
timepoint                       0.453    0.035 12.941 0.000  0.384  0.521
hx_before_preg___1             

## Plug & Play Hypothesis Block

In [116]:
outcomes = {'Total_M3_01' : 3, 'TotalBISQ_M12_01' : 12}
predictors = ['hx_before_preg___1', 'hx_before_preg___4', 'hx_during_preg___1', 'hx_during_preg___4']
covariates = ['Mage', 'ses']

In [118]:
percent_non_null(df, predictors + list(outcomes.keys()) + covariates)

25.87% rows are fully non-null across all specified columns

Per-column non-null percentages:

hx_before_preg___1                 :  95.45%
hx_before_preg___4                 :  95.45%
hx_during_preg___1                 :  95.35%
hx_during_preg___4                 :  95.35%
Total_M3_01                        :  33.07%
TotalBISQ_M12_01                   :  36.76%
Mage                               :  98.67%
ses                                :  98.03%


25.87

In [ ]:
results, df_long = build_mixed_model(df = df, 
                  outcomes_dict = outcomes,
                  predictor_cols = predictors,
                  covariate_cols = covariates,
                  outcome_label = 'TotalBISQ')  